# 1. 环境配置

## 1.1 python 环境准备

In [2]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3 arxiv==2.3.1

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [3]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 实践代码

为了能够顺利的演示内置中间件的使用详情，这里我们使用一段简单的智能体代码演示：

In [ ]:
from langchain_community.chat_models import ChatTongyi
import os
llm = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-turbo")

from langchain_community.agent_toolkits.load_tools import load_tools
tools = load_tools(["arxiv"])

from langgraph.checkpoint.memory import InMemorySaver 
memory = InMemorySaver()

from langchain.agents import create_agent
agent = create_agent(model=llm, 
                     tools=tools, 
                     system_prompt="You are a helpful assistant", 
                     checkpointer=memory)


论文编号 1605.08386 的信息如下：

- **发表日期**：2016-05-26
- **标题**：Heat-bath random walks with Markov bases
- **作者**：Caprice Stanley, Tobias Windisch
- **摘要**：研究了由有限允许移动集生成的格点图，这些移动可以是任意长度。我们证明了在固定整数矩阵的纤维上的这些图的直径可以从上面由一个常数界。然后研究了这些图上的热浴随机游走的混合行为。还给出了移动集的显式条件，使得热浴随机游走（Glauber动力学的一种推广）在固定维度下是一个扩展器。


# 2. 创建中间件（MiddleWare）

## 2.1 中间件简介

传统 ReAct Agent 的核心循环虽然已经能实现「推理—行动—观察」的闭环，但在实际生产中，我们常常需要更细粒度地控制模型调用和工具调用的过程（例如：日志记录、异常捕获、安全过滤、动态上下文注入等）。

因此，LangChain v1.0 在基于 create_agent() 的基础上，在底层引入了 Middleware（中间件）机制 —— 让开发者可以在智能体执行的每一个关键阶段「前后挂钩」。

比如在模型调用前处理输入、在工具调用后拦截输出，从而真正实现可观测、可调试、可扩展的智能体执行流程。

我们可以把 MiddleWare 中间件想象为是一个“钩子系统（hook system）”，可以在智能体每一步执行前后插入自定义逻辑。常见的可插入位置包括：
- agent 开始前后：before_agent/after_agent
- 模型调用前后：before_model/after_model
- 工具调用时：wrap_tool_call
- 模型调用时：wrap_model_call

这些可插入位置整体可以分为两类：
- Node-style hooks：是在固定节点的执行前或执行后运行的函数。其特点是不会拦截执行流程，假如有返回值会合并进当前的 state，并且可以提前终止执行（jump_to）。
- Wrap-style hooks：是真正的拦截器，因为其包裹着模型或工具的执行逻辑，并且拥有完全控制权。我们可以通过 handler(request) 来决定要不要调用、何时调用、调用几次。并且可以改输入、改输出、处理异常，甚至可以完全替换整个执行逻辑。

在 LangChain 中，假如我们要自定义中间件，通常有两种写法。一种是“装饰器式（Decorator-based）”。另一种是“类式（Class-based）”。

## 2.2 装饰器式（Decorator-based）

装饰器式中间件等于我们写一个普通的 Python 函数，然后用官方提供的装饰器（比如 @before_model , @after_model , @wrap_tool_call）把这个函数注册成一个“中间件钩子（hook）”。这个方法主要针对某一个特定的插入位置实现，适用于比较简单的场景中。

In [5]:
from langchain.agents.middleware import before_model, AgentState
from langgraph.runtime import Runtime

@before_model
def log_before_model(state: AgentState, runtime: Runtime):
    print(f"📤 调用模型前的消息数量: {len(state['messages'])}")

agent = create_agent(model=llm, 
 tools=tools, 
 system_prompt="You are a helpful assistant", 
 middleware=[log_before_model], # 添加中间件
 checkpointer=memory)

## 2.2 类式（Class-based）

类式中间件是“面向对象风格的钩子集合”。其允许我们定义一个继承自 AgentMiddleware 的类，这个类本身就能够去定义 before_model，after_model 等方法。然后在这个类里实现任意多个生命周期方法（Hook），这些方法在 Agent 的不同阶段自动被调用。我们传入一个中间件类就等于同时传入多个装饰器式中间件，因此适用于比较复杂的场景中。

In [6]:
from langchain.agents.middleware import AgentMiddleware

class MyMiddleware(AgentMiddleware):
    def before_model(self, state, runtime):
        ...
    def after_model(self, state, runtime):
        ...

agent = create_agent(model=llm, 
 tools=tools, 
 system_prompt="You are a helpful assistant", 
 middleware=[MyMiddleware()],
 checkpointer=memory)

其实前面的装饰器式的方式从底层来说也是生成了一个 AgentMiddleware 子类。比如：

In [7]:
@before_model
def log_before_model(state, runtime):
    print(f"模型即将调用，消息数 {len(state['messages'])}")

LangChain 内部其实执行了类似的代码：

In [8]:
class LogBeforeModel(AgentMiddleware):
    def before_model(self, state, runtime):
        print(f"模型即将调用，消息数 {len(state['messages'])}")

所以我们写装饰器版本，本质上就是写“匿名类中间件”。

# 3. Node-style hooks

Node-style hooks 包括：
- agent 开始前后：before_agent/after_agent
- 模型调用前后：before_model/after_model

对于 Node-style hooks 而言，其整体的输入和输出的参数都是一致的，所以这里我们就打包一起来进行展示：
- 输入参数：state（当前 Agent 的完整状态）, runtime（当前运行环境对象）
- 输出内容：None（不更新状态），dict（更新状态）

## 3.1 输入参数 AgentState （动态信息）

AgentState 是一个用于管理 Agent 当前状态的结构（Schema）。它保存智能体在每个时间点上需要记住的所有上下文信息。其默认的结构是下面这样的：

```python
{"messages": [...],
  "thread_model_call_count": 1,
  "run_model_call_count": 1}
```

那所有的上下文记忆都会存在 messages 之中。

假如我们想要添加一些自定义的字段到 AgentState中，我们需要先定义一个类：

In [9]:
from langchain.agents.middleware import AgentState

class CustomAgentState(AgentState):
    user_id: str
    preferences: dict

这个类是继承原本的 AgentState 的，所以会把原本的内容都保留。然后我们可以在 create_agent 的时候通过 state_schema 参数将其加入进去：

In [10]:
agent = create_agent(
 model=llm,
 tools=tools,
 state_schema=CustomAgentState,
 checkpointer=InMemorySaver(),
)

并在调用时进行传入，此时打印的内容中就可以看到传入的相关信息了：

In [11]:
result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "Hello"}],
        "user_id": "user_123",  
        "preferences": {"theme": "dark"}  
    },
    {"configurable": {"thread_id": "1"}})

print(result)
print(result["user_id"])
print(result["preferences"])

{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='e50b53b8-aa97-47e8-8d80-84b7d6af8f60'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': 'eec3ac6b-a0da-47d6-aeb4-ba0bb17f1c64', 'token_usage': {'input_tokens': 206, 'output_tokens': 9, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 215}}, id='lc_run--019b2fc3-b236-7960-b009-63d0dc53c321-0')], 'user_id': 'user_123', 'preferences': {'theme': 'dark'}}
user_123
{'theme': 'dark'}


## 3.2 输入参数 Runtime （静态信息）

Runtime 是 LangChain 在每次 agent 执行期间传递上下文信息的对象，通常用于存放用户信息、数据库连接、长期记忆等运行所需资源。
其内部存放着三类的信息：
- .context：用户信息（自行在 create_agent() 中定义的）
- .store：长期记忆的相关信息
- .stream：流失输出相关信息

```python
Runtime(context=Context(
    user_id="user_123",
    role="admin"
  ),
  store=<RedisStore>,
  stream=<CustomStreamWriter>)
```

假如我们希望自己来定义 runtime.context 的话，我们首先需要自定义一个有 user_name 字段的类，并且标明其对于的格式信息（使用 dataclass 装饰器定义结构）。

In [12]:
from dataclasses import dataclass

@dataclass
class Context:
    user_name: str

然后在创建 agent 时进行绑定：

In [13]:
agent = create_agent(
 model=llm,
 tools=tools,
 context_schema=Context, # ✅ 这里指定结构
 checkpointer=InMemorySaver(),
)

但是我们直接打印调用结果的话是没办法看到 Runtime 的，因为里面只会显示 AgentState 的动态信息：

In [14]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    config={"configurable": {"thread_id": "2"}},
    context=Context(user_name="John Smith") # ✅ 这里传入实例
)

print(result)

{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='893c416d-34df-44d3-9500-c5b2f2bac9c7'), AIMessage(content="I don't have access to personal information about individual users. If you have any questions or need assistance, feel free to ask!", additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': 'f0a1bfa3-3b18-43c9-82c4-7ad0c7c1fc45', 'token_usage': {'input_tokens': 210, 'output_tokens': 26, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 236}}, id='lc_run--019b2fc3-b6fa-76b3-8000-5f7a1229d965-0')]}


所以我们假如想要使用的话就需要添加中间件进行展示：

In [15]:
@before_model
def log_before_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:  
    print(f"Processing request for user: {runtime.context.user_name}")  
    return None

agent = create_agent(
 model=llm,
 tools=tools,
 context_schema=Context, # ✅ 这里指定结构
 checkpointer=InMemorySaver(),
 middleware=[log_before_model],
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    config={"configurable": {"thread_id": "2"}},
    context=Context(user_name="John Smith") # ✅ 这里传入实例
)

Processing request for user: John Smith


或在 Agent 工具调用时进行使用，当然这就要求我们在定义工具的时候就必须把 runtime 参数载入进去：

In [16]:
from langchain.tools import tool, ToolRuntime  

@tool
def fetch_user_email_preferences(runtime: ToolRuntime[Context]) -> str:  
    """Fetch the user's email preferences from the store."""
    user_id = runtime.context.user_id  

    preferences: str = "The user prefers you to write a brief and polite email."
    if runtime.store:  
        if memory := runtime.store.get(("users",), user_id):  
            preferences = memory.value["preferences"]

    return preferences

## 3.3 输出返回值 None | dict

Node-style 的核心机制在于：返回一个字典用于“更新 Agent 的状态”或“改变执行流程”。

它的返回值可以是：
- None —— 表示不做任何更改，继续执行；
- dict[str, Any] —— 用于修改当前的 Agent 状态（就是前面提到的 AgentState），比如增加 message、更新变量、甚至直接跳出流程。

在 Node-style hook 中，如果返回的字典包含某个字段，就会覆盖原来的值；如果没返回该字段，就继续保留 AgentState 中已有的值不变。

常见的返回字典格式为：

```python
{
    "messages": [AIMessage("内容")],    # 更新消息列表，替换掉原有消息（如果设置 jump_to）
    "jump_to": "end",                  # 跳转到某个节点，如 "end"、"tools"、"model"
    "custom_key": "value",             # 自定义状态键值对（如果使用了自定义 AgentState）
}
```

## 3.4 Node-style hooks 创建案例

下面我们就来用几个真实的案例来展示一下如何创建 Node-style Hooks 的 MiddleWare 。

### 3.4.1 案例 1：log_before_model
下面这个中间件是一个作用在模型前（before_model），其作用也非常简单，就是去打印一下当前的历史记录中有多少条信息。

所以这里我们通过 state['messages'] 的方式将整个历史记录的信息进行获取，然后通过 len() 函数去记录对应历史记录信息的数量（每一条 HumanMessages 或者 AIMessage 都算是列表中的一条信息），并最后将其打印出来。

In [17]:
from typing import Any
from langchain.agents import AgentState, create_agent
from langchain.agents.middleware import before_model
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.runtime import Runtime  

@before_model
def log_before_model(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    print(f"About to call model with {len(state['messages'])} messages")
    return None

agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=InMemorySaver(),
    middleware=[log_before_model],
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    config={"configurable": {"thread_id": "1"}}
)

print(result)

About to call model with 1 messages
{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='3dbdea0f-ed3f-4a57-b412-ecdda29a82d6'), AIMessage(content="I don't have access to your personal information. You'll need to provide your name for me to know it.", additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '02ea6d1b-356b-4928-b074-7421870a348d', 'token_usage': {'input_tokens': 210, 'output_tokens': 23, 'prompt_tokens_details': {'cached_tokens': 128}, 'total_tokens': 233}}, id='lc_run--019b2fc3-bb26-7302-92d5-a51229999652-0')]}


### 3.4.2 案例 2：validate_output

而 validate_output 这个中间件的作用位置时在模型输出后（after_model），其主要目的是审查模型的输出里是否包含关键字“BLOCKED“（系统提示词中进行设置）。

主要使用的方法也是从 AgentState 中获取完整的历史记录列表 messages，然后获取里面最后一条（刚模型调用完也就是 AIMessage）里的内容 content ，看里面是否包含关键字。假如包含的话就更新信息（dict）返回不能回应并跳转到结束 end，否则就无事发生（None）。

In [18]:
from langchain.agents.middleware import after_model
from langchain.messages import AIMessage

@after_model(can_jump_to=["end"])
def validate_output(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    last_message = state["messages"][-1]
    if "BLOCKED" in last_message.content:
        return {"messages": [AIMessage("I cannot respond to that request.")],
            "jump_to": "end"}
    return None

system_prompt = """
You are a safety-controlled assistant.
Your single task is to check whether the user's message contains harmful, dangerous, illegal, violent, self-harm, hateful, criminal, or unethical content.

If the user's question or instruction is unsafe, sensitive, or prohibited in any way,
your ONLY output must be:

BLOCKED

Do NOT provide explanations, warnings, apologies, or alternative suggestions.

If the user's message is safe and harmless, answer normally.
"""

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=InMemorySaver(),
    middleware=[validate_output], # 写入中间件
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "how to destroy a building?"}]},
    config={"configurable": {"thread_id": "1"}}
)

print(result["messages"][-1].content)

I cannot respond to that request.


类似的，假如我们要用类装饰器实现信息的截获，可以按以下方式实现。略微不同的是前面 @after_model 中可以传入 can_jump_to 参数，而这里的话可以添加 @hook_config 来让 LangGraph 知道我们要跳转的位置（在类创建方法里，不添加就是任意位置都能去）。

In [19]:
from langchain.agents.middleware import AgentMiddleware, hook_config, AgentState
from langchain.messages import AIMessage
from langgraph.runtime import Runtime
from typing import Any

class BlockedContentMiddleware(AgentMiddleware):
    @hook_config(can_jump_to=["end"])
    def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        last_message = state["messages"][-1]
        if "BLOCKED" in last_message.content:
            return {
                "messages": [AIMessage("I cannot respond to that request.")],
                "jump_to": "end"
            }
        return None

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=InMemorySaver(),
    middleware=[BlockedContentMiddleware()], # 替换前面的函数式中间件
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "how to destroy a building?"}]},
    config={"configurable": {"thread_id": "1"}}
)

print(result["messages"][-1].content)

I cannot respond to that request.


### 3.4.3 案例 3：多中间件

在类装饰器里，我们可以同时设置模型开始前后的中间件：
- 比如模型开始前我们和之前一样打印一下当天聊天记录中信息的数量
- 模型开始后我们打印一下模型输出的内容（最新一条也就是 AIMessage）
- 两者都只是信息的打印，而不进行流程或信息的更新（return None）

In [20]:
from langchain.agents.middleware import AgentMiddleware, AgentState
from langgraph.runtime import Runtime
from typing import Any

class LoggingMiddleware(AgentMiddleware):
    def before_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        print(f"About to call model with {len(state['messages'])} messages")
        return None

    def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        print(f"Model returned: {state['messages'][-1].content}")
        return None

agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=InMemorySaver(),
    middleware=[LoggingMiddleware()], # 替换前面的中间件
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "你好，介绍一下自己吧"}]},
    config={"configurable": {"thread_id": "2"}}
)

About to call model with 1 messages
Model returned: 你好！我是Qwen，是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我可以帮助你回答问题、创作文字、逻辑推理、编程等。如果你有任何问题或需要帮助，随时告诉我！


### 3.4.4 类装饰器属性添加

当然类装饰器里可以加入一些更复杂的东西，比如我们可以添加一个历史记录的限制。这里首先我们在类中调用父类的初始化方法 super().__init__() 并自定义了一个属性 max_messages 用于后续的数量判定。然后在实际的调用中，假如超过了这个设置的数量的话，就更新信息并跳转到结尾 end。在实际创建 agent 时，我们可以把这个类以及属性的值也给写入，这样就可以进行应用了。

In [21]:
class MessageLimitMiddleware(AgentMiddleware):
    def __init__(self, max_messages: int = 50):
        super().__init__()
        self.max_messages = max_messages
    @hook_config(can_jump_to=["end"]) 
    def before_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        if len(state["messages"]) >= self.max_messages:
            return {
                "messages": [AIMessage("Conversation limit reached.")],
                "jump_to": "end"
            }
        return None
    
agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=InMemorySaver(),
    middleware=[MessageLimitMiddleware(max_messages=2)], # 添加属性
)

result1 = agent.invoke(
    {"messages": [{"role": "user", "content": "你是谁呀，200字介绍一下"}]},
    config={"configurable": {"thread_id": "1"}}
)

result2 = agent.invoke(
    {"messages": [{"role": "user", "content": "你能做什么？"}]},
    config={"configurable": {"thread_id": "1"}}
)

print(result1["messages"][-1].content)
print(result2["messages"][-1].content)

我是一个大型语言模型，由阿里云开发。我的名字是通义千问，我可以帮助用户回答问题、创作文字、逻辑推理、编程等。我的目标是为用户提供高质量的服务，帮助他们更好地完成工作和学习。如果你有任何问题或需要帮助，随时可以问我！
Conversation limit reached.


### 3.4.5 自定义 Agent State

除此之外我们还可以添加自定义的 state 进去。这个时候传入的类就不是 AgentMiddleware ，而是 AgentMiddleware[CustomState]，并且写入时也要 state_schema = CustomState。

In [22]:
from langchain.agents.middleware import AgentState, AgentMiddleware
from typing_extensions import NotRequired
from typing import Any

class CustomState(AgentState):
    model_call_count: NotRequired[int]
    user_id: NotRequired[str]

class CallCounterMiddleware(AgentMiddleware[CustomState]):
    state_schema = CustomState
    
    @hook_config(can_jump_to=["end"])
    def before_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        count = state.get("model_call_count", 0)
        user_id = state.get("user_id", "Unknown")
        if count >= 1:
            return {
                "messages": [AIMessage(content=f"{user_id} BLOCKED: model call limit exceeded.")],
                "jump_to": "end"
            }
        return None

    def after_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        return {"model_call_count": state.get("model_call_count", 0) + 1}
    


然后这里我们通过 agent.invoke 的内容将 messages, model_call_count, user_id 等参数进行写入，然后在模型运行后通过 @after_model 进行 model_call_count 动态的更新。这样我们就能够在 before_model 获取模型运行次数，假如大于一次就跳转到 END 处。

In [23]:
agent = create_agent(
    model=llm,
    middleware=[CallCounterMiddleware()],
    checkpointer=InMemorySaver(),
    tools=tools,
)

result1 = agent.invoke(
    {"messages": [{"role": "user", "content": "你是谁"}],
    "model_call_count": 0,
    "user_id": "user-123",},
    config={"configurable": {"thread_id": "10"}}
)

result2 = agent.invoke(
    {"messages": [{"role": "user", "content": "你能做什么"}]},
    config={"configurable": {"thread_id": "10"}}
)

print(result1["messages"][-1].content)
print(result2["messages"][-1].content)

我是一个AI助手，旨在帮助您解答各种问题和提供相关信息。如果您有任何问题或需要帮助，请随时告诉我！
user-123 BLOCKED: model call limit exceeded.


当然调用的时候不写入的话也会自动创建的，默认起始的数量为 0 且 user_id 未 Unknown。

In [24]:
agent = create_agent(
    model=llm,
    middleware=[CallCounterMiddleware()],
    checkpointer=InMemorySaver(),
    tools=tools,
)

result1 = agent.invoke(
    {"messages": [{"role": "user", "content": "你是谁"}]},
    config={"configurable": {"thread_id": "10"}}
)

result2 = agent.invoke(
    {"messages": [{"role": "user", "content": "你能做什么"}]},
    config={"configurable": {"thread_id": "10"}}
)

print(result1["messages"][-1].content)
print(result2["messages"][-1].content)

我是通义千问，由阿里巴巴集团旗下的通义实验室研发的超大规模语言模型。我可以帮助你解答问题、创作文字、编程、逻辑推理等各种任务。你可以问我任何问题，我会尽力提供帮助。
Unknown BLOCKED: model call limit exceeded.


## 4 Wrap-Style hooks

Wrap-style hooks 包括：
- 工具调用时：wrap_tool_call
- 模型调用时：wrap_model_call

对于 wrap_tool_call 和 wrap_model_call 而言，虽然两者的输入参数和输出内容很类似，但是整体还有一些不同：
- 输入参数：request（模型工具调用请求 ModelRequest 或工具调用请求 ToolCallRequest），handler（模型或工具调用函数）
- 输出内容：对于 wrap_model_call 输出的是 ModelResponse。而对于 wrap_tool_call 而言输出的可能是 ToolMessage （直接运行获取结果），也可能是 Command （等待人类反馈后执行下一步操作）

因此下面我们将分别来进行剖析与讲解：

## 4.1 Wrap_model_call

### 4.1.1 输入参数 ModelRequest

在 Wrap-style hook 中，request 表示的是当前模型调用/工具调用的完整请求对象。

它是一个封装了输入信息、上下文信息、运行状态的结构体，你可以在这里读取、修改甚至替换掉整个请求内容，再决定是否要继续调用、如何调用。

对于 ModelResponse 而言，其内部包含的内容如下：

```python
ModelRequest(
    model: BaseChatModel,                    # ✅ 当前使用的模型实例
    messages: list[BaseMessage],             # ✅ 对话历史（用户、助手、系统消息）
    system_prompt: Optional[str] = None,     # ✅ 系统指令（部分模型用作 system role）
    tool_choice: Optional[Union[str, dict]] = "auto",  # ✅ 工具调用策略
    tools: Optional[list[BaseTool]] = None,  # ✅ 可用的工具列表
    response_format: Optional[str] = "text", # ✅ 返回格式：text / json（可选）
    state: AgentState,                                 # 运行过程中的动态参数
    runtime: Runtime[ContextT],                 # 运行过程中的静态信息 
    model_settings: Optional[dict] = {},     # ✅ 传给模型的附加参数，如温度、max_tokens
)
```

对于 ModelRequest 的内容，其实我们也是能够修改后再传给模型的。我们只需要使用 .override() 方法进行修改即可。

比如我们可以用 .override() 创建新请求再调用 handler：

```python
new_request = request.override(
    model_settings={"temperature": 0.1},
    system_prompt="你是一个极简风格中文助手")
response = handler(new_request)
```

我们甚至可以这样做来组合多个 override：

```python
request = request.override(messages=filtered_messages)
request = request.override(model=new_model)
response = handler(request)
```

### 4.1.2 输入参数 handler

在 Wrap_model_call 里，我们其实可以看到还传入了一个参数 handler，这个 handler 的类型是一个 Callable[[ModelRequest], ModelResponse]，这意味着说 handler 是一个 LangChain 内置好的函数，我们只需要输入 ModelRequest， 就会输出 ModelResponse。

这个函数可能是下面这样：

```python
def handler(request: ModelRequest) -> ModelResponse:
    # 1. 组装 prompt
    prompt = build_prompt_from(request.messages, request.system_prompt)
    # 2. 从 model_settings 里取参数
    temperature = request.model_settings.get("temperature", 0.7)
    # 3. 调用模型
    response_text = request.model.invoke(prompt=prompt, temperature=temperature)
    # 4. 包装成标准格式
    return ModelResponse(result=[AIMessage(content=response_text)])
```

### 4.1.3 输出内容 ModelResponse

ModelResponse 是大语言模型完成一次推理之后，LangChain Agent 接收到的结果封装。

其字段结构如下：

```python
ModelResponse(
    result: list[BaseMessage],         # 模型生成的消息列表（主要是 AI 回复）
    structured_response: Any = None    # 若使用结构化响应，解析后的 JSON 对象
)
```

### 4.1.4 wrap_model_call 创建案例

#### 案例 1：retry_model

对于 Wrap-style hooks 这类的中间件，主要解决并再是信息的审查类型的问题了。而是对实际调用前后可能出现的一些状况进行准备。

比如在模型调用过程中，我们最怕的就是模型调用时出现网络波动，导致无法成功连接上模型出现报错。那下面的 retry_model 函数其实就是让大模型在调用时假如出现报错的话有三次重启的机会，假如三次都出现错误才显示报错信息。

In [25]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def retry_model(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    for attempt in range(3):
        try:
            return handler(request)
        except Exception as e:
            if attempt == 2:
                raise
            print(f"Retry {attempt + 1}/3 after error: {e}")
            
            
agent = create_agent(
    model=llm,
    middleware=[retry_model],
    checkpointer=InMemorySaver(),
    tools=tools,
)

假如我们通过类进行创建的话，就可以在传入到 agent 时来自定义设置重试的次数 max_retries（默认值为 3 ）。

如果运行成功的话，就直接执行模型调用的操作。但如果出现报错并超过指定次数的话，则打印并显示报错信息：

In [26]:
class RetryMiddleware(AgentMiddleware):
    def __init__(self, max_retries: int = 3):
        super().__init__()
        self.max_retries = max_retries
    def wrap_model_call(self, request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
        for attempt in range(self.max_retries):
            try:
                return handler(request)
            except Exception as e:
                if attempt == self.max_retries - 1:
                    raise
                print(f"Retry {attempt + 1}/{self.max_retries} after error: {e}")
                
agent = create_agent(
    model=llm,
    middleware=[RetryMiddleware()],
    checkpointer=InMemorySaver(),
    tools=tools,
)

result1 = agent.invoke(
    {"messages": [{"role": "user", "content": "你是谁"}]},
    config={"configurable": {"thread_id": "10"}}
)

result2 = agent.invoke(
    {"messages": [{"role": "user", "content": "你能做什么"}]},
    config={"configurable": {"thread_id": "10"}}
)

print(result1["messages"][-1].content)
print(result2["messages"][-1].content)

我是Qwen，是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字、逻辑推理、编程等多种任务。如果你有任何问题或需要帮助，欢迎随时告诉我！
我能够回答问题、创作文字、逻辑推理、编程等多种任务。无论是需要帮助解决复杂的问题，还是想要创作故事、邮件、公文等文字内容，或是进行数据分析和编程，我都可以提供支持。如果你有任何具体的需求或问题，欢迎随时告诉我！


#### 案例 2：DynamicModelMiddleware 多模型选择

除了重试模型以外，我们在调用模型的时候还可以考虑根据用户的提问的轮数来更改使用的模型。比如像下面的代码中，我们根据 request 中的 messages 列表（里面存放了之前的对话记录）来决定是使用 qwen-max 模型还是 qwen-turbo 模型。这里实现的方式就是通过修改 request.model 内的信息来实现的。

In [27]:
class DynamicModelMiddleware(AgentMiddleware):
  def wrap_model_call(self, request: ModelRequest, handler):
    if len(request.messages) > 3:
      request.model = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-turbo")
    else:
      request.model = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-max")
    return handler(request)

agent = create_agent(
    model=llm,
    middleware=[DynamicModelMiddleware()],
    checkpointer=InMemorySaver(),
    tools=tools,
)

result = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386"}]}, config={"configurable": {"thread_id": "10"}})

result = agent.invoke(
    {"messages": [{"role": "user", "content": "你是谁"}]},
    config={"configurable": {"thread_id": "10"}}
)

print(result["messages"][1].response_metadata["model_name"])
print(result["messages"][-1].response_metadata["model_name"])

C:\Users\76391\AppData\Local\Temp\ipykernel_27364\3124932934.py:6: DeprecationWarning: Direct attribute assignment to ModelRequest.model is deprecated. Use request.override(model=...) instead to create a new request with the modified attribute.
  request.model = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-max")
C:\Users\76391\AppData\Local\Temp\ipykernel_27364\3124932934.py:6: DeprecationWarning: Direct attribute assignment to ModelRequest.model is deprecated. Use request.override(model=...) instead to create a new request with the modified attribute.
  request.model = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-max")
C:\Users\76391\AppData\Local\Temp\ipykernel_27364\3124932934.py:4: DeprecationWarning: Direct attribute assignment to ModelRequest.model is deprecated. Use request.override(model=...) instead to create a new request with the modified attribute.
  request.model = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="

qwen-max
qwen-turbo


但是这种方法已经被丢弃了，所以我们需要使用新的 .override() 的方法实现。

LangChain v1.0 开始，ModelRequest 被设计为不可变对象（immutability），为了保证链路稳定与可追踪性，官方强制推荐使用 .override() 的方法相当于是重新创建一个新的 request ，并且我们还可以加上一些参数来限制 max_messages 的数量：

In [28]:
class DynamicModelMiddleware(AgentMiddleware):
    def __init__(self, max_messages: int = 3):
        super().__init__()
        self.max_messages = max_messages
    def wrap_model_call(self, request: ModelRequest, handler):
        if len(request.messages) > self.max_messages:
            new_model = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-turbo")
        else:
            new_model = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-max")
        new_request = request.override(model=new_model)
        return handler(new_request)

agent = create_agent(
    model=llm,
    middleware=[DynamicModelMiddleware(max_messages=4)],
    checkpointer=InMemorySaver(),
    tools=tools,
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386"}]}, 
    config={"configurable": {"thread_id": "10"}})

result = agent.invoke(
    {"messages": [{"role": "user", "content": "你是谁"}]},
    config={"configurable": {"thread_id": "10"}})

print(result["messages"][1].response_metadata["model_name"])
print(result["messages"][-1].response_metadata["model_name"])

qwen-max
qwen-turbo


这样我们正常的进行调用就会发现第二次调用的模型就从原本的 qwen-max 转变为了 qwen-trubo ，这样就可以大大节省 token 的消耗了。

#### 案例 3：dynamic_prompt

除了传统的 wrap_model_call 以外，在 LangChain 官方文档中还介绍了一个基于其改造的装饰器 dynamic_prompt。相比于 wrap_model_call，dynamic_prompt 仅仅作用在模型调用前，所以其传入的只有 request 的信息而没有 handler。

所以从技术的角度上看，@dynamic_prompt 是基于 @wrap_model_call 的特例封装（专用于 system_prompt 动态注入）。

下面这个例子中，我们通过 get() 获取 request 里的 runtime.context 信息，看里面是否有保存和 user_id 的内容，假如有就用里面的内容。假如没有就用默认的 guest 写入提示词中。

In [29]:
from langchain.agents.middleware import dynamic_prompt

@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    user_name = getattr(request.runtime.context, "user_name", "guest")
    system_prompt = f"你是 {user_name} 大佬的小助手，记好了哦！"
    return system_prompt

from dataclasses import dataclass
@dataclass
class Context:
    user_name: str

agent = create_agent(
 model=llm,
 tools=tools,
 middleware=[dynamic_system_prompt],
 context_schema=Context, # ✅ 这里指定结构
 checkpointer=InMemorySaver(),
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "你是谁？"}]},
    config={"configurable": {"thread_id": "1"}},
    context=Context(user_name="剑锋") # ✅ 这里传入实例
)

print(result["messages"][-1].content)

我是剑锋大佬的小助手，有什么可以帮您的吗？


## 4.2 Wrap_tool_call

### 4.2.1 输入参数 ToolCallRequest

与 ModelRequest 类似， ToolCallRequest 是在模型触发工具调用（Function Calling） 之后，传入给 wrap_tool_call 这个 hook 的参数对象。其内部包含的参数如下：

```python
ToolCallRequest(
    tool_call: ToolCall,              # ✅ 工具调用信息（包含 name、args、id）
    tool: Optional[BaseTool] = None,  # ✅ 当前注册的工具（未注册则为 None）
    state: Any,                       # ✅ 当前智能体状态（通常是 AgentState 字典）
    runtime: Optional[ToolRuntime] = None  # ✅ 工具运行时上下文
)
```

#### ToolRuntime

ToolRuntime 是 LangChain 在工具调用过程中自动注入的运行环境对象，它为工具函数提供了对智能体当前执行上下文（state/context/config）的访问能力，帮助我们实现更灵活、更智能的工具行为。

```python
ToolRuntime(
  state: dict | BaseModel,    # 🧠 AgentState 当前智能体状态（短期记忆）
  context: dict | None,     # 🌐 Runtime Context 执行上下文（运行时传入）
  config: RunnableConfig,    # ⚙️ 当前执行配置，包含 run_id、metadata、tags 等
  stream_writer: StreamWriter,  # 📡 实时流式输出对象
  tool_call_id: str | None,   # 🔗 当前工具调用唯一标识符
  store: BaseStore | None    # 💾 Store 持久化存储接口（长期记忆）
)
```

#### 更新 ToolCallRequest

类似于 ModelRequest ，ToolCallRequest 的内容也是能够修改后再传给模型的。我们也是使用 .override() 方法进行修改即可。

比如我们也可以用 .override() 创建新请求再调用 handler：

```python
# 你想修改工具参数，比如把城市改成“深圳”
new_args = {"city": "深圳"}
new_tool_call = request.tool_call.override(args=new_args)

# 使用 override 创建新请求副本
new_request = request.override(tool_call=new_tool_call)

# 同时修改多个内容构建 ToolCallRequest
new_request = request.override(
    tool_call=new_tool_call,     # 修改调用内容
    state=new_state,             # 替换 agent state
)
```



### 4.2.2 输入参数 handler

与 Wrap_model_call 类似，Wrap_tool_call 本质上也是一个 Callable 的函数（Callable[[ToolCallRequest], ToolMessage | Command]）

其输入的是 ToolCallRequest， 返回的可能是 ToolMessage 或 Command。

一个可能的 handler 内部包括以下几个流程：
- 从 Function Calling 返回的信息中获取必要信息
- 校验工具是否存在
- 执行工具
- 将结果封装为 ToolMessage

```python
def handler(request: ToolCallRequest) -> ToolMessage:
    # 1. 提取工具调用相关信息
    tool_call = request.tool_call           # 获取模型的调用信息
    tool = request.tool                     # 获取已注册的工具

    # 2. 校验工具是否存在（可选）
    if tool is None:
        raise ValueError(f"工具 {tool_call.name} 未注册")

    # 3. 执行工具（** 解包参数）
    try:
        result = tool.invoke(**tool_call.args)
    except Exception as e:
        result = f"工具执行失败: {str(e)}"

    # 4. 封装为 ToolMessage
    return ToolMessage(
        tool_call_id=tool_call.id,
        content=str(result)  # 工具返回值必须是字符串
    )
```

### 4.2.3 输出内容 ToolMessage

当一个工具被调用后，LangChain / LangGraph 会把该工具的执行结果，包装成一个 ToolMessage，然后把这个 ToolMessage 追加到对话历史里，交还给大模型。

这个时候 ToolMessage 的内部如下所示：

```python
ToolMessage(
    tool_call_id: str,                        # ✅ 必填，对应模型发起的某个 tool 调用的唯一 ID
    content: Union[str, list[str | dict]],    # ✅ 工具执行结果，发回模型的主要内容（会强制转成 str 或 list）
    type: Literal["tool"] = "tool",           # ✅ 固定值："tool"，表示这是一条工具返回消息
    artifact: Any = None,                     # ⛳ 可选，工具的完整原始产出（通常不发给模型）
    status: Literal["success", "error"] = "success",  # ⛳ 可选，执行状态
    additional_kwargs: dict = {},             # ⛳ 预留字段，当前未使用
    response_metadata: dict = {},             # ⛳ 预留字段，当前未使用
)
```

### 4.2.4 输出内容 Command

Command 是 LangGraph 里一个“指令对象”，节点返回它之后，可以同时做两件事：
- 更新状态（state）
- 决定下一步跳到哪个节点

常见的使用方式如下：
- update={...}：把这些键值写回全局状态，别的节点之后就能用到。
- goto="show_dashboard"：下一步别按默认连线走了，直接去跑 show_dashboard 这个节点。

```python
def check_login(state: dict) -> Command[Literal["show_dashboard"]]:
    # 假设我们验证登录成功了
    return Command(
        update={"user": "alice", "login_ok": True},  # ① 更新状态
        goto="show_dashboard"                        # ② 指定下一个节点
    )
```

Command 不只是“去固定下一个节点”，还可以“根据状态决定去哪个节点”，比如：

``` python
def route_question(state: dict) -> Command[Literal["faq_bot", "human_agent"]]:
    # 如果是简单问题 -> 机器人回答
    if state["question_type"] == "FAQ":
        return Command(
            update={"handled_by": "bot"},
            goto="faq_bot"
        )
    # 否则 -> 转人工客服
    else:
        return Command(
            update={"handled_by": "human"},
            goto="human_agent"
        )
```

 -> Command[Literal["faq_bot", "human_agent"]] 这里表示可能跳转的节点

除了最常见的 update 和 goto 以外，Command 还有两个其他的参数 graph 和 resume：

- graph 常用于多智能体/多子流程协作。比如当前只是某个子任务在进行，完成后需要回到主流程中时需要使用：

```python
def refund_agent(state: dict) -> Command[Literal["handoff_to_main"]]:
    # 子图里的节点
    return Command(
        update={"refund_status": "submitted"},
        goto="handoff_to_main",  # 交还回主流程里的节点
        graph=Command.PARENT     # 👈 关键：告诉 LangGraph，我现在要跳回父图
    )
```

- resume 常用于人类介入时使用。当人类介入后，先是会有一个暂停点（interrupt）中断流程，然后再接收反馈（resume） 后重启。后续讲解 Human-In-Loop 会深入讲解。

### 4.2.5 wrap_tool_call 案例展示

#### 案例 1：monitor_tool

而对于工具调用而言，其实和模型调用一样，主要是查看一下调用的工具是否是正确的，或者说当工具调用出现问题的时候来重试几次甚至换个工具。

那对于下面的中间件 monitor_tool ，其先是打印了一下使用的工具和传入的参数信息，然后通过 handler 去调用了一下。假如成功就打印信息返回结果，假如失败就报错。

In [30]:
from langchain.agents.middleware import wrap_tool_call
from langchain.tools.tool_node import ToolCallRequest
from langchain.messages import ToolMessage
from langgraph.types import Command
from typing import Callable

@wrap_tool_call
def monitor_tool(request: ToolCallRequest, handler: Callable[[ToolCallRequest], ToolMessage | Command]) -> ToolMessage | Command:
    print(f"Executing tool: {request.tool_call['name']}")
    print(f"Arguments: {request.tool_call['args']}")
    try:
        result = handler(request)
        print(f"Tool completed successfully")
        return result
    except Exception as e:
        print(f"Tool failed: {e}")
        raise

agent = create_agent(
    model=llm,
    middleware=[monitor_tool],
    checkpointer=InMemorySaver(),
    tools=tools,
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386"}]}, 
    config={"configurable": {"thread_id": "10"}})


Executing tool: arxiv
Arguments: {'query': '1605.08386'}
Tool completed successfully


类似的，这个中间件我们也可以通过类进行创建，区别就是名称改为了 ToolMonitoringMiddleware 而已，并且传入给 middleware 时要写为 middleware=[ToolMonitoringMiddleware()] 。其他在实际使用上其实没有任何区别。

In [31]:
class ToolMonitoringMiddleware(AgentMiddleware):
    def wrap_tool_call(self, request: ToolCallRequest, handler: Callable[[ToolCallRequest], ToolMessage | Command]) -> ToolMessage | Command:
        print(f"Executing tool: {request.tool_call['name']}")
        print(f"Arguments: {request.tool_call['args']}")

        try:
            result = handler(request)
            print(f"Tool completed successfully")
            return result
        except Exception as e:
            print(f"Tool failed: {e}")
            raise
        
agent = create_agent(
    model=llm,
    middleware=[ToolMonitoringMiddleware()],
    checkpointer=InMemorySaver(),
    tools=tools,
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386"}]}, 
    config={"configurable": {"thread_id": "10"}})

Executing tool: arxiv
Arguments: {'query': '1605.08386'}
Tool completed successfully
